In [ ]:
"""
Note: Add more detailed description to the final .py file!

This is a master script for running various versions of the sliding window
coupling lag model for the hyperscanning project. There are four main "modes"
in which this can be run.

(0) DEBUG MODE: Run on a super truncated version of the dataset with short
modeling windows and a small number of permutations to make sure nothing
breaks.
(1) FULL CONCATENATION MODE: Concatenate all speech turns into one long time
series, run the model on this concatenated time series and ignore the option
to implement a rolling window scheme. This is a special case of the rolling
window approach where the window size is the total number of TRs in the
storytelling task and the number of rolling window steps is 1.
(2) MODEL EACH TURN MODE: Run the coupling model separately for each speech
turn and then average parameter estimates across turns. This is a special
case of the rolling window analysis where both the window and step size are
equal to the number of TRs per speech turn.
(3) CUSTOM ROLLING WINDOW MODE: manually set rolling window parameters

Other parameters can be tweaked in the "Parameter setup" section.

Note that the term "TRs" is used somewhat loosely below, as the time series we
are working with were generated by interpolating across the original EPI time
series at slightly different time points compared to the original TRs. However,
despite the interpolation step, we maintain the total number of TRs from the
original time series. So while the samples don't perfectly map onto the original
TRs per se... it's close enough...
"""


In [1]:
import os
import sys
sys.path.append('/dartfs-hpc/rc/lab/W/WheatleyT/f00589z/hyperscanning/support_scripts/')
from couplingISC import *
from CircleShift_3 import *
from phaseScramble_2 import *
from coupling_lm_permTest_v2 import *
import numpy as np
import pandas as pd
import math
import scipy.io as sio
import matplotlib.pyplot as plt
from numpy.random import random_sample
import time
from joblib import Parallel, delayed
import statsmodels.stats.multitest as multi
import pickle
from nilearn import image as nImage
from nilearn import input_data
from nilearn import datasets
from nilearn import surface
from nilearn import plotting
%matplotlib inline
# %load_ext filprofiler

/dartfs-hpc/rc/home/z/f00589z/.conda/envs/hypeScanCentral/lib/python3.9/site-packages/nilearn/datasets/__init__.py:87: FutureWarning: Fetchers from the nilearn.datasets module will be updated in version 0.9 to return python strings instead of bytes and Pandas dataframes instead of Numpy arrays.
  warn("Fetchers from the nilearn.datasets module will be "


In [2]:
startTime = time.time()

In [3]:
# save output?
saveOutput = True

# select participants to include
dbicSubs = list(range(1,9)) # dbic participants to include
cbsSubs = list(range(1,9)) # cbs participants to include

# kind of hacky but predefining the total number of TRs that will be in each concatenated time series
totTRs = 615

# number of TRs per turn
TRsPerTurn = 41

# number of speech turns per participant per condition
numTurns = round(totTRs / TRsPerTurn)

# get speaker/listener TR indices
turnTRs = [[]] * numTurns
for TURN in range(numTurns):
    if TURN == 0:
        inds = np.array(list(range(TRsPerTurn)))
    else:
        inds = inds + TRsPerTurn
    turnTRs[TURN] = inds

# also kind of hacky but predefining the number of voxels
totalVox = 69880

# set some joblib parameters
numJobs = 32 # number of parallel processes for joblib to use
verbosity = 0 # joblib verbosity

# set analysis parameters
maxT = 6 # max TR lag to include in linear models
permuts = 1000 # number of permutations to use for pair-wise permutation tests
numChunks = 7 # number of voxel chunks
voxelCoords = list(range(0,totalVox))
padding = 'mean' # mean or zero
pseudo = False # if True, compute coupling ISC for pseudopairs
parallel = True # if True, use joblib to run ISC in parallel across permutations
alpha = 0.05 # permutation test alpha level
circShift = False # True = use circle shifting for TR shuffling, False = use phase scrambling
shuffleRange = 41 # Number of TRs over which to implement shuffling. If zero, use entire time series.

#################################
### Rolling window parameters ###
#################################

# set rolling window mode to 1 of 4 options:
# (0) DEBUG MODE:
# (1) FULL CONCATENATION MODE: Concatenate all speech turns into one long time
# series, run the model on this concatenated time series and ignore the option
# to implement a rolling window scheme. This is a special case of the rolling
# window approach where the window size is the total number of TRs in the
# storytelling task and the number of rolling window steps is 1.
# (2) MODEL EACH TURN MODE: Run the coupling model separately for each speech
# turn and then average parameter estimates across turns. This is a special
# case of the rolling window analysis where both the window and step size are
# equal to the number of TRs per speech turn.
# (3) CUSTOM ROLLING WINDOW MODE: manually set rolling window parameters
rollMode = 0

# Set rolling window parameters based on MODE selected above
if rollMode == 3: # CUSTOM ROLLING WINDOW MODE

    # set custom parameters
    winTRs = 308 # window size [TRs]
    stepSize = 307 # step size [TRs]
    numSteps = 1 # number of steps to use if maxCoverage is set to False below [integer]

    # set whether or not to let the selected window and step sizes cover the
    # maximum number of available TRs. Note that if maxCoverage == True, numSteps
    # will be overridden and set to the maximum number of steps determined below.
    maxCoverage = True

    # compute maximum possible number of steps based on winTR and stepSize
    maxSteps = int(math.ceil((totTRs - winTRs + 1) / stepSize))
    if maxCoverage: # if max coverage selected...
        numSteps = maxSteps # redefine numSteps to maximum possible number of steps

    # find any TRs that will be missed given winTRs, stepSize, and maxSteps
    lastTr = winTRs + (numSteps - 1) * stepSize # last TR this approach will analyze
    TRsLeftOut = totTRs - lastTr # number of TRs that will be left out by the current approach

    # feedback
    print('\n*** INITIATING CUSTOM ROLLING WINDOW MODE ***')
    print('Rolling window size: ' + str(winTRs) + ' TRs')
    print('Step size: ' + str(stepSize) + ' TRs')
    print('Number of steps: ' + str(numSteps))

    if TRsLeftOut > 0: # if any TRs will end up getting left out based on window and step sizes...
        print('WARNING! The last ' + str(TRsLeftOut) + ' TRs of the storytelling task will be left out due to the selected window and step sizes!')
    elif TRsLeftOut <= 0:
        print('WARNING! The inputted rolling window parameters assume a larger time series than we have! Something needs to change!')

elif rollMode == 1: # FULL CONCATENATION MODE

    # set parameters
    winTRs = totTRs # use all TRs
    stepSize = np.nan # no step size as there is only the one window
    numSteps = 1 # just one step

    # feedback
    print('\n*** INITIATING FULL CONCATENATION MODE ***')
    print('Running coupling model on single time series concatenated from')
    print('all speech turns of a given speaker-listener orientation.')

elif rollMode == 2: # MODEL EACH TURN MODE

    # set parameters
    winTRs = 41 # use all TRs
    stepSize = 41 # no step size as there is only the one window
    numSteps = 15 # 615 total TRs / 41 TRs per speech turn = 15 steps

    # feedback
    print('\n*** INITIATING MODEL EACH TURN MODE ***')
    print('Running the coupling model separately for each speech turn and then')
    print('averaging parameter estimates across turns.')

elif rollMode == 0: # DEBUG MODE

    # set parameters so we just look at two super short windows
    winTRs = 20 # window size [TRs]
    stepSize = 20 # step size [TRs]
    numSteps = 2 # number of steps to use if maxCoverage is set to False below [integer]

    # feedback
    print('\n*** INITIATING DEBUG MODE ***')
    print('Running the coupling model for ' + str(numSteps) + ' ' + str(winTRs) + '-TR windows.')

    # reset some additional parameters to make things quick
    permuts = 10
    numChunks = 2
    maxT = 2

# get TRs to analyze in each step of rolling window analysis
stepTRs = [[]] * numSteps
for STEP in range(numSteps):
    if STEP == 0:
        stepTRs[STEP] = np.array(list(range(winTRs)))
    else:
        stepTRs[STEP] = stepTRs[STEP-1] + stepSize

# check the shuffleRange variable, reset if necessary, and provide some feedback
if shuffleRange > winTRs:
    print('\nThe inputted shuffling range of '+  str(shuffleRange)  + ' TRs is greater than the width')
    print('of the modeling window (' + str(winTRs) + ' TRs). This will not work. Defaulting to')
    print('shuffling across the entire modeling window.')
    shuffleRange = 0 # reset shuffle range to default of entire rolling window
else:
    if shuffleRange == 0 or shuffleRange == winTRs:
        print('\nShuffling across the entire modeling window of ' + str(winTRs) + ' TRs.')
    elif 0 < shuffleRange < winTRs:
        print('\nShuffling across ' + str(shuffleRange) + '-TR subsets of the modeling window of ' + str(winTRs) + ' TRs.')
if shuffleRange == TRsPerTurn:
    print('\nNote that this means we are shuffling at the level of speech turns.')


*** INITIATING DEBUG MODE ***
Running the coupling model for 2 20-TR windows.

The inputted shuffling range of 41 TRs is greater than the width
of the modeling window (20 TRs). This will not work. Defaulting to
shuffling across the entire modeling window.


In [4]:
# seed the RNG
rngSeed = 8675309
rng = np.random.default_rng(rngSeed)

# mark starting time
startTime = time.time()

# load pair and run data
pairsAndRuns = pd.read_csv(r'/dartfs-hpc/rc/lab/W/WheatleyT/f00589z/hyperscanning/misc/hyperscanning_pair_and_run_lookup.csv')
print(pairsAndRuns)

# set data folder
dataFolder = '/dartfs-hpc/rc/lab/W/WheatleyT/f00589z/hyperscanning/storytelling/parseEPI_output_files/'

# set output folder
outputFolder = '/dartfs-hpc/rc/lab/W/WheatleyT/f00589z/hyperscanning/storytelling/lag_model_output/'

# get numbers of pairs
realPairN = len(list(set(dbicSubs) & set(cbsSubs)))
pseudoPairN = len(dbicSubs)*len(cbsSubs) - realPairN
totalPairN = realPairN + pseudoPairN

# set column names for 'pairMap' dataframe to be used as a reference for pair information
pairMap_header = ['dbicNum', #DBIC subject number
                  'dbicID', #DBIC subject ID (e.g., sid000007)
                  'cbsNum', #CBS subject number
                  'cbsID', #CBS subject ID
                  'pairType', #1=real, 0=pseudo
                  'condition', #1=joint, 0=independent
                  'dbicSpeaker', #1=DBIC speaker, CBS listener, 0=CBS speaker, DBIC listener
                  'sFile', #speaker file name
                  'lFile', #listener file name
                  'duration'] #ISC duration [min]

# number of rows to preallocate for pairMap dataframe
numRows = (realPairN + pseudoPairN) * 4

# preallocate pair map data frame
pairMap = pd.DataFrame(columns = pairMap_header, index=range(numRows))

# set order in which to load tasks for DBIC and CBS subs within each pair
dTask = ['listener','speaker','listener','speaker']
cTask = ['speaker','listener','speaker','listener']

# condition strings for feedback at top of ROW loop below
storyConds = ['independent','joint','joint minus independent']
pairTypes = ['pseudo','real']
speakers = ['CBS','DBIC']

   pair       DBIC        CBS  ind_run  joint_run
0     1  sid000014  hid000001        2          1
1     2  sid000007  hid000002        2          1
2     3  sid000009  hid000003        2          1
3     4  sid000560  hid000004        1          2
4     5  sid000535  hid000005        1          2
5     6  sid000102  hid000006        2          1
6     7  sid000416  hid000007        2          1
7     8  sid000499  hid000008        1          2
8     9  sid000142  hid000009        1          2


In [5]:
# initialize pairMap row counter
ROW = 0

# for each DBIC subject...
for dbicPairN in dbicSubs:

    # get DBIC sub ID and run info
    dbicSub = pairsAndRuns['DBIC'][dbicPairN] # e.g., 'sid000007'
    dbicRunInd = pairsAndRuns['ind_run'][dbicPairN] # current subject's independent run #
    dbicRunJoint = pairsAndRuns['joint_run'][dbicPairN] # current subject's joint run #
    dRun = [dbicRunInd, dbicRunInd, dbicRunJoint, dbicRunJoint] # make an array of independent and joint run #s to simplify loading below

    # get dbic file names
    dbicFiles = [[]]*4
    for FILE in range(4):
        dbicFiles[FILE] = dataFolder + 'sub-' + dbicSub + '_ses-pair0' + str(dbicPairN + 1) + '_task-storytelling' + \
            str(dRun[FILE]) + '_run-0' + str(dRun[FILE]) + '_bold_space-MNI152NLin2009cAsym_preproc_nuisRegr_2021_' + dTask[FILE] + '.mat'  # [TRs x voxels]

    # for each CBS subject...
    for cbsPairN in cbsSubs:

        # get CBS sub ID and run info
        cbsSub = pairsAndRuns['CBS'][cbsPairN] # e.g., 'hid000002'
        cbsRunInd = pairsAndRuns['ind_run'][cbsPairN]
        cbsRunJoint = pairsAndRuns['joint_run'][cbsPairN]
        cRun = [cbsRunInd, cbsRunInd, cbsRunJoint, cbsRunJoint] # make an array of independent and joint run #s to simplify loading below

        for FILE in range(4):

            # get dbic file names
            cbsFiles = [[]] * 4
            cbsFiles[FILE] = dataFolder + 'sub-' + cbsSub + '_ses-pair0' + str(cbsPairN + 1) + '_task-storytelling' + \
                             str(cRun[FILE]) + '_run-0' + str(cRun[FILE]) + \
                             '_bold_space-MNI152NLin2009cAsym_preproc_nuisRegr_2021_' + cTask[FILE] + '.mat'  # [TRs x voxels]

            ##############################
            ### index stuff to pairMap ###
            ##############################

            # subject numbers and IDs
            pairMap['dbicNum'][ROW] = dbicPairN + 1
            pairMap['dbicID'][ROW] = dbicSub
            pairMap['cbsNum'][ROW] = cbsPairN + 1
            pairMap['cbsID'][ROW] = cbsSub

            # pair type
            if pairMap['dbicNum'][ROW] == pairMap['cbsNum'][ROW]:
                pairMap['pairType'][ROW] = 1 #real pair
            else:
                pairMap['pairType'][ROW] = 0 #pseudo pair

            # speaker/listener
            if ROW % 2: # if it's an odd row - DBIC speaker, CBS listener
                pairMap['dbicSpeaker'][ROW] = 1
                pairMap['sFile'][ROW] = dbicFiles[FILE] # speaker file name
                pairMap['lFile'][ROW] = cbsFiles[FILE] # listener file name
            else: # if it's an even row - CBS speaker, DBIC listener
                pairMap['dbicSpeaker'][ROW] = 0
                pairMap['sFile'][ROW] = cbsFiles[FILE] # speaker file name
                pairMap['lFile'][ROW] = dbicFiles[FILE] # listener file name

            # independent/joint condition
            if FILE < 2:
                pairMap['condition'][ROW] = 0 # independent
            else:
                pairMap['condition'][ROW] = 1 # joint

            ROW += 1

In [6]:
if not pseudo: # if we are not including pseudopairs...
    pairMap = pairMap.loc[pairMap['pairType'] == 1,:] # remove pseudopairs
    pairMap = pairMap.reset_index(drop=True)
    pseudoStr = 'excluding' # set string to use below
else:
    pseudoStr = 'including' # set string to use below

# print final pairMap dataframe
pairMap

,dbicNum,dbicID,cbsNum,cbsID,pairType,condition,dbicSpeaker,sFile,lFile,duration
0,2,sid000007,2,hid000002,1,0,0,/dartfs-hpc/rc/lab/W/WheatleyT/f00589z/hypersc...,/dartfs-hpc/rc/lab/W/WheatleyT/f00589z/hypersc...,NaN
1,2,sid000007,2,hid000002,1,0,1,/dartfs-hpc/rc/lab/W/WheatleyT/f00589z/hypersc...,/dartfs-hpc/rc/lab/W/WheatleyT/f00589z/hypersc...,NaN
2,2,sid000007,2,hid000002,1,1,0,/dartfs-hpc/rc/lab/W/WheatleyT/f00589z/hypersc...,/dartfs-hpc/rc/lab/W/WheatleyT/f00589z/hypersc...,NaN
3,2,sid000007,2,hid000002,1,1,1,/dartfs-hpc/rc/lab/W/WheatleyT/f00589z/hypersc...,/dartfs-hpc/rc/lab/W/WheatleyT/f00589z/hypersc...,NaN
4,3,sid000009,3,hid000003,1,0,0,/dartfs-hpc/rc/lab/W/WheatleyT/f00589z/hypersc...,/dartfs-hpc/rc/lab/W/WheatleyT/f00589z/hypersc...,NaN
5,3,sid000009,3,hid000003,1,0,1,/dartfs-hpc/rc/lab/W/WheatleyT/f00589z/hypersc...,/dartfs-hpc/rc/lab/W/WheatleyT/f00589z/hypersc...,NaN
6,3,sid000009,3,hid000003,1,1,0,/dartfs-hpc/rc/lab/W/WheatleyT/f00589z/hypersc...,/dartfs-hpc/rc/lab/W/WheatleyT/f00589z/hypersc...,NaN
7,3,sid000009,3,hid000003,1,1,1,/dartfs-hpc/rc/lab/W/WheatleyT/f00589z/hypersc...,/dartfs-hpc/rc/lab/W/WheatleyT/f00589z/hypersc...,NaN
8,4,sid000560,4,hid000004,1,0,0,/dartfs-hpc/rc/lab/W/WheatleyT/f00589z/hypersc...,/dartfs-hpc/rc/lab/W/WheatleyT/f00589z/hypersc...,NaN
9,4,sid000560,4,hid000004,1,0,1,/dartfs-hpc/rc/lab/W/WheatleyT/f00589z/hypersc...,/dartfs-hpc/rc/lab/W/WheatleyT/f00589z/hypersc...,NaN


In [7]:
if rollMode == 0:
    if pairMap.shape[0] > 8: # if there are more than 8 rows in pairMap (i.e., two pairs)
        pairMap = pairMap.iloc[range(8),:] # limit pairMap to just the first two pairs
print('Using a truncated version of pairMap with only two pairs because we are in DEBUG MODE.')
pairMap

Using a truncated version of pairMap with only two pairs because we are in DEBUG MODE.


,dbicNum,dbicID,cbsNum,cbsID,pairType,condition,dbicSpeaker,sFile,lFile,duration
0,2,sid000007,2,hid000002,1,0,0,/dartfs-hpc/rc/lab/W/WheatleyT/f00589z/hypersc...,/dartfs-hpc/rc/lab/W/WheatleyT/f00589z/hypersc...,NaN
1,2,sid000007,2,hid000002,1,0,1,/dartfs-hpc/rc/lab/W/WheatleyT/f00589z/hypersc...,/dartfs-hpc/rc/lab/W/WheatleyT/f00589z/hypersc...,NaN
2,2,sid000007,2,hid000002,1,1,0,/dartfs-hpc/rc/lab/W/WheatleyT/f00589z/hypersc...,/dartfs-hpc/rc/lab/W/WheatleyT/f00589z/hypersc...,NaN
3,2,sid000007,2,hid000002,1,1,1,/dartfs-hpc/rc/lab/W/WheatleyT/f00589z/hypersc...,/dartfs-hpc/rc/lab/W/WheatleyT/f00589z/hypersc...,NaN
4,3,sid000009,3,hid000003,1,0,0,/dartfs-hpc/rc/lab/W/WheatleyT/f00589z/hypersc...,/dartfs-hpc/rc/lab/W/WheatleyT/f00589z/hypersc...,NaN
5,3,sid000009,3,hid000003,1,0,1,/dartfs-hpc/rc/lab/W/WheatleyT/f00589z/hypersc...,/dartfs-hpc/rc/lab/W/WheatleyT/f00589z/hypersc...,NaN
6,3,sid000009,3,hid000003,1,1,0,/dartfs-hpc/rc/lab/W/WheatleyT/f00589z/hypersc...,/dartfs-hpc/rc/lab/W/WheatleyT/f00589z/hypersc...,NaN
7,3,sid000009,3,hid000003,1,1,1,/dartfs-hpc/rc/lab/W/WheatleyT/f00589z/hypersc...,/dartfs-hpc/rc/lab/W/WheatleyT/f00589z/hypersc...,NaN


In [8]:
# get unique pairs
uniquePairs = pairMap[["dbicID", "cbsID"]].drop_duplicates()
uniquePairs = uniquePairs.reset_index(drop=True)

# get independent vs joint row indices for each pair
pairInds = [[]] * uniquePairs.shape[0]
for PAIR in range(len(pairInds)):

    pairInds[PAIR] = [[]] * 2
    for COND in [0,1]:
        pairInds[PAIR][COND] = pairMap[(pairMap['dbicID'] == uniquePairs['dbicID'][PAIR]) & (pairMap['cbsID'] == uniquePairs['cbsID'][PAIR]) & (pairMap['condition'] == COND)].index.tolist()

# print list of unique pairs
uniquePairs

,dbicID,cbsID
0,sid000007,hid000002
1,sid000009,hid000003


In [9]:
# preallocate chunk indices list
chunkInds = [[]] * numChunks

# get main chunk size (if the number of chunks doesn't go into totalVox evenly there will be one remainder chunk of smaller size)
chunkSize = np.ceil(totalVox / numChunks)

# get chunk indices
for CHUNK in range(numChunks):
    if CHUNK == 0: # if it's the first chunk...

        # get initial indices from 0 to the main chunk size
        chunkInds[CHUNK] = np.arange(chunkSize).astype(int)

    elif CHUNK == numChunks and not totalVox / numChunks == 0: # if it's the last chunk and it's a remainder chunk

        # get remainder chunk indices
        remainder = totalVox - chunkSize * (numChunks-1)
        chunkInds[CHUNK] = (chunkInds[CHUNK-1] + chunkSize)[range(remainder)].astype(int)

    else:

        # add chunk size to previous chunk indices
        chunkInds[CHUNK] = (chunkInds[CHUNK-1] + chunkSize).astype(int)

# preallocate voxel chunk array for duration
chunkDur = [[]] * numChunks
pairData_by_step = [[]] * numChunks
pairData_across_steps = [[]] * numChunks
pairData_jVSi_by_step = [[]] * numChunks
pairData_jVSi_across_steps = [[]] * numChunks
groupData_by_step = [[]] * numChunks
groupData_across_steps = [[]] * numChunks

In [10]:
# %%filprofile

# initial feedback
print('\nRunning lag models with ' + str(len(np.unique(pairMap['dbicNum']))) + ' pairs, ' + pseudoStr + ' pseudo pairs')
print('Number of voxel chunks: ' + str(numChunks))
print('Permutations: ' + str(permuts))
print('Max TR lag: ' + str(maxT))

# FOR EACH CHUNK OF VOXELS
for CHUNK in range(numChunks):

    # voxel chunk feedback
    print('\n*** CHUNK ' + str(CHUNK+1) + ' OF ' + str(numChunks) + ': VOXELS ' + str(np.min(chunkInds[CHUNK])+1) + ' to ' + str(np.max(chunkInds[CHUNK])+1) + ' ***\n')

    ######################
    ### Coupling model ###
    ######################

    # initalize lists with length equal to the total number of combinations
    # of pair, speaker-listener relationship, and joint vs independent condition
    # (i.e., the number of rows in pairMap)
    iscData = [[]] * pairMap.shape[0]
    perms = [[]] * pairMap.shape[0]

    # if we're using more than one modeling window, preallocate lists for cross-window parameter averages
    if numSteps > 1:
        paramAvg = [[]] * pairMap.shape[0]
        perms_paramAvg = [[]] * pairMap.shape[0]
    else:
        paramAvg = np.nan
        perms_paramAvg = np.nan

    # for each combination of pair, speaker-listener relationship, and joint vs independent condition...
    for ROW in range(pairMap.shape[0]):

        # get current time so we can keep track of the time it takes to analyze pair data
        # from each row of pairMap
        rowStart = time.time()

        # initialize iscData subArray for each window in the series of rolling ISC windows...
        iscData[ROW] = [[]] * numSteps
        perms[ROW] = [[]] * numSteps

        # pair/condition-wise feedback
        print('estimating lag model C' + str(CHUNK + 1) + 'R' + str(ROW + 1) + ' of C' + str(numChunks) + 'R' + str(pairMap.shape[0]))
        print('DBIC participant: ' + pairMap['dbicID'][ROW])
        print('CBS participant: ' + pairMap['cbsID'][ROW])
        print('storytelling condition: ' + storyConds[pairMap['condition'][ROW]])
        print('pair type: ' + pairTypes[pairMap['pairType'][ROW]])
        print('speaker: ' + speakers[pairMap['dbicSpeaker'][ROW]] + '\n')

        # load speaker timeseries
        sFile = pairMap['sFile'][ROW]
        if os.path.isfile(sFile): # if there is a file to load...
            dummyFile = sio.loadmat(sFile) # load file
            if pairMap['dbicSpeaker'][ROW] == 1:
                speaker = dummyFile['dbicSpeaker'][:,voxelCoords] # get timeseries data
            else:
                speaker = dummyFile['cbsSpeaker'][:,voxelCoords] # get timeseries data
            del dummyFile
        else:
            print('WARNING! ' + sFile + ' not found!')

        # load listener timeseries
        lFile = pairMap['lFile'][ROW]
        if os.path.isfile(lFile):
            dummy = sio.loadmat(lFile) # load data
            if pairMap['dbicSpeaker'][ROW] == 1:
                listener = dummy['cbsListener'][:,voxelCoords] # get time series data
            else:
                listener = dummy['dbicListener'][:,voxelCoords] # get time series data
            del dummy
        else:
            print('WARNING! ' + lFile + ' not found!')

        # subset to current voxel chunk
        speaker = speaker[:,chunkInds[CHUNK]]
        listener = listener[:,chunkInds[CHUNK]]
        numVox = len(chunkInds[CHUNK])

        # for each modeling window...
        for STEP in range(numSteps):

            # initialize dictionaries for ISC variables
            iscData[ROW][STEP] = {'betas':np.empty([numVox,maxT*2+1]),
                                  'resids':np.empty(numVox),
                                  'R':np.empty(numVox),
                                  'R_pVals':dict(),
                                  'R_masks':dict(),
                                  'R_pSig':dict()}
            perms[ROW][STEP] = {'betas':np.empty([numVox,maxT*2+1,permuts]),
                                'resids':np.empty([numVox,permuts]),
                                'R':np.empty([numVox,permuts])}

            # get subset of TRs corresponding to current rolling window step
            s_trunc = speaker[stepTRs[STEP],:]
            l_trunc = listener[stepTRs[STEP],:]

            # run real coupling model

            iscData[ROW][STEP]['betas'], iscData[ROW][STEP]['resids'], iscData[ROW][STEP]['R'] = coupling_onestep(s_trunc,l_trunc,maxT,padding)

            ########################
            ### permutation test ###
            ########################

            # if not running parallel
            if not parallel:

                # for each permutation...
                for PERM in range(permuts):

                    # run coupling model with scrambled time series
                    if circShift: # circle shifting...
                        perms[ROW][STEP]['betas'][:,:,PERM], perms[ROW][STEP]['resids'][:,[PERM]], perms[ROW][STEP]['R'][:,[PERM]] = coupling_onestep(Circle_Shift_3(s_trunc,shuffleRange=shuffleRange),l_trunc,maxT,padding)
                    else: # phase scrambling...
                        perms[ROW][STEP]['betas'][:,:,PERM], perms[ROW][STEP]['resids'][:,[PERM]], perms[ROW][STEP]['R'][:,[PERM]] = coupling_onestep(phase_scrambling_2(s_trunc,shuffleRange=shuffleRange),l_trunc,maxT,padding)

            # if running ISC in parallel across permutations
            else:

                # run coupling model with scrambled time series with permutations in parallel
                if circShift: # circle shifting...
                    tmp = Parallel(n_jobs=numJobs, verbose=verbosity)(delayed(coupling_onestep)
                                                  (Circle_Shift_3(s_trunc,shuffleRange=shuffleRange),
                                                   l_trunc,
                                                   maxT,
                                                   padding)
                                                  for PERM in range(permuts))
                else: # phase scrambling
                    tmp = Parallel(n_jobs=numJobs, verbose=verbosity)(delayed(coupling_onestep)
                                                  (phase_scrambling_2(s_trunc,shuffleRange=shuffleRange),
                                                   l_trunc,
                                                   maxT,
                                                   padding)
                                                  for PERM in range(permuts))

                # unpack joblib output
                for PERM in range(permuts):
                    perms[ROW][STEP]['betas'][:,:,PERM] = tmp[PERM][0] # beta coefficients
                    perms[ROW][STEP]['resids'][:,[PERM]] = tmp[PERM][1] # residuals
                    perms[ROW][STEP]['R'][:,[PERM]] = tmp[PERM][2] # R^2



            # run permutation test
            iscData[ROW][STEP]['R_pVals'], iscData[ROW][STEP]['R_masks'], iscData[ROW][STEP]['R_pSig'] = coupling_lm_permTest_v2(iscData[ROW][STEP]['R'], perms[ROW][STEP]['R'], alpha)

            # print proportion of voxels with signficant FDR-corrected R^2 values at each step
            print('Step ' + str(STEP+1) + ' of ' + str(numSteps) + ': TRs ' + str(np.min(stepTRs[STEP])) + ' to ' + str(np.max(stepTRs[STEP])))
            print('% of voxels with signficant FDR-corrected R^2 values: ' + str(round(iscData[ROW][STEP]['R_pSig']['two'] * 100,2)) + '%')

        # if using more than one modeling window...
        if numSteps > 1:

            # initialize dictionaries for parameter estimate averages
            paramAvg[ROW] = {'betas':np.empty([numVox,maxT*2+1]),
                             'resids':np.empty(numVox),
                             'R':np.empty(numVox),
                             'R_pVals':dict(),
                             'R_masks':dict(),
                             'R_pSig':dict()}
            perms_paramAvg[ROW] = {'betas':np.empty([numVox,maxT*2+1,permuts]),
                                   'resids':np.empty([numVox,permuts]),
                                   'R':np.empty([numVox,permuts])}

            # get averages parameter estimates across modeling windows
            paramAvg[ROW]['betas'] = np.mean([iscData[ROW][STEP]['betas'] for STEP in range(numSteps)], axis=0) # mean beta coefficients
            paramAvg[ROW]['resids'] = np.mean([iscData[ROW][STEP]['resids'] for STEP in range(numSteps)], axis=0) # mean residuals
            paramAvg[ROW]['R'] = np.mean([iscData[ROW][STEP]['R'] for STEP in range(numSteps)], axis=0) # mean R^2
            perms_paramAvg[ROW]['betas'] = np.mean([perms[ROW][STEP]['betas'] for STEP in range(numSteps)], axis=0) # permuted beta coefficients
            perms_paramAvg[ROW]['resids'] = np.mean([perms[ROW][STEP]['resids'] for STEP in range(numSteps)],axis=0) # permuted residuals
            perms_paramAvg[ROW]['R'] = np.mean([perms[ROW][STEP]['R'] for STEP in range(numSteps)],axis=0) # permuted mean R^2

            # run permutation test on averaged parameter estimates
            paramAvg[ROW]['R_pVals'], paramAvg[ROW]['R_masks'], paramAvg[ROW]['R_pSig'] = coupling_lm_permTest_v2(paramAvg[ROW]['R'], perms_paramAvg[ROW]['R'], alpha)

            # print proportion of voxels with signficant FDR-corrected R^2 values
            print('\nAcross steps:')
            print('% of voxels with signficant FDR-corrected mean R^2 values: ' + str(round(paramAvg[ROW]['R_pSig']['two'] * 100,2)) + '%')

        # get duration for current row
        pairMap['duration'][ROW] = round((time.time() - rowStart) / 60,2)
        print('processing time for model ' + str(ROW+1) + ': ~' + str(pairMap['duration'][ROW]) + ' min\n')

    ######################################################################################
    ### Get lag model computation duration and mean processing time per row of pairMap ###
    ######################################################################################

    lagComp_total = round(time.time() - startTime / 60, 2) # [min]
    lagComp_mean_per_pairMap_row = pairMap['duration'].mean()

    ####################################################################################################
    ### get joint - independent contrast for each unique pair (averaging across speaker orientation) ###
    ####################################################################################################

    # preallocate array for unique pairs
    jVSi = [[]] * len(pairInds)
    perms_jVSi = [[]] * len(pairInds)
    if numSteps > 1:
        jVSi_paramAvg = [[]] * len(pairInds)
        perms_jVSi_paramAvg = [[]] * len(pairInds)
    else:
        jVSi_paramAvg = np.nan
        perms_jVSi_paramAvg = np.nan

    # for each unique pair...
    for PAIR in range(len(pairInds)):

        # preallocate step-wise lists
        jVSi[PAIR] = [[]] * numSteps
        perms_jVSi[PAIR] = [[]] * numSteps

        # for each modeling window...
        for STEP in range(numSteps):

            # preallocate arrays for mean R^2 values across speaker orientations
            mean_R = [[]] * 2 # real mean
            mean_R_perms = [[]] * 2 # permutation test means

            # for each condition (0=independent, 1=joint)...
            for COND in [0,1]:

                # average across speaker orientation
                mean_R[COND] = np.mean([iscData[pairInds[PAIR][COND][0]][STEP]['R'], iscData[pairInds[PAIR][COND][1]][STEP]['R']], axis=0)
                mean_R_perms[COND] = np.mean([perms[pairInds[PAIR][COND][0]][STEP]['R'], perms[pairInds[PAIR][COND][1]][STEP]['R']], axis=0)

            # initialize dictionaries (joint - independent) permutation tests
            jVSi[PAIR][STEP] = {'R':np.empty(numVox),
                                'R_pVals':dict(),
                                'R_masks':dict(),
                                'R_pSig':dict()}
            perms_jVSi[PAIR][STEP] = {'R':np.empty([numVox,permuts])}

            # get difference scores
            jVSi[PAIR][STEP]['R'] = mean_R[1] - mean_R[0]
            perms_jVSi[PAIR][STEP]['R'] = mean_R_perms[1] - mean_R_perms[0]

            # run permutation test
            jVSi[PAIR][STEP]['R_pVals'], jVSi[PAIR][STEP]['R_masks'], jVSi[PAIR][STEP]['R_pSig'] = coupling_lm_permTest_v2(jVSi[PAIR][STEP]['R'], perms_jVSi[PAIR][STEP]['R'], alpha)

        # if using more than one modeling window...
        if numSteps > 1:

            # preallocate arrays for mean R^2 values across speaker orientations
            mean_R = [[]] * 2 # real mean
            mean_R_perms = [[]] * 2 # permutation test means

            # for each condition (0=independent, 1=joint)...
            for COND in [0,1]:

                # average across speaker orientation
                mean_R[COND] = np.mean([paramAvg[pairInds[PAIR][COND][0]]['R'], paramAvg[pairInds[PAIR][COND][1]]['R']], axis=0)
                mean_R_perms[COND] = np.mean([perms_paramAvg[pairInds[PAIR][COND][0]]['R'], perms_paramAvg[pairInds[PAIR][COND][1]]['R']], axis=0)

            # initialize dictionaries (joint - independent) permutation tests
            jVSi_paramAvg[PAIR] = {'R':np.empty(numVox),
                                   'R_pVals':dict(),
                                   'R_masks':dict(),
                                   'R_pSig':dict()}
            perms_jVSi_paramAvg[PAIR] = {'R':np.empty([numVox,permuts])}

            # get difference scores
            jVSi_paramAvg[PAIR]['R'] = mean_R[1] - mean_R[0]
            perms_jVSi_paramAvg[PAIR]['R'] = mean_R_perms[1] - mean_R_perms[0]

            # run permutation test
            jVSi_paramAvg[PAIR]['R_pVals'], jVSi_paramAvg[PAIR]['R_masks'], jVSi_paramAvg[PAIR]['R_pSig'] = coupling_lm_permTest_v2(jVSi_paramAvg[PAIR]['R'], perms_jVSi_paramAvg[PAIR]['R'], alpha)


    ###################################################
    ### Group level stats for conditions separately ###
    ###################################################

    # preallocate
    condInds = [[]] * 2 # arrays for pairMap row indices for each of the independent and joint conditions
    groupData = [[]] * 3 # condition-speicific arrays for stepwise permutation tests at the group level
    if numSteps > 1:
        groupData_paramAvg = [[]] * 3 # condition-specific arrays for permutation tests on average parameters (across steps) at the group level
    else:
        groupData_paramAvg = np.nan

    # for each condition (0=independent, 1=joint, 2=joint minus independent)
    for COND in range(3):

        # get row indices from pairMap corresponding to each condition
        if COND < 2:
            condInds[COND] = np.where(pairMap['condition'] == COND)[0]

        # preallocate
        groupData[COND] = [[]] * numSteps

        for STEP in range(numSteps):

            # initialize dictionary
            groupData[COND][STEP] = {'R_median':np.empty(numVox),
                                     'R_median_pVals':dict(),
                                     'R_median_masks':dict(),
                                     'R_median_pSig':dict(),
                                     'R_mean':np.empty(numVox),
                                     'R_mean_pVals':dict(),
                                     'R_mean_masks':dict(),
                                     'R_mean_pSig':dict()}

            # if independent or joint condition...
            if COND < 2:

                # compute means and medians
                groupData[COND][STEP]['R_median'] = np.median([iscData[ROW][STEP]['R'] for ROW in condInds[COND]], axis=0)
                groupData[COND][STEP]['R_mean'] = np.mean([iscData[ROW][STEP]['R'] for ROW in condInds[COND]], axis=0)
                perms_median = np.median([perms[ROW][STEP]['R'] for ROW in condInds[COND]], axis=0)
                perms_mean = np.mean([perms[ROW][STEP]['R'] for ROW in condInds[COND]], axis=0)

            # if joint minus independent condition...
            else:

                # compute means and medians
                groupData[COND][STEP]['R_median'] = np.median([jVSi[PAIR][STEP]['R'] for PAIR in range(len(jVSi))], axis=0)
                groupData[COND][STEP]['R_mean'] = np.mean([jVSi[PAIR][STEP]['R'] for PAIR in range(len(jVSi))], axis=0)
                perms_median = np.median([perms_jVSi[PAIR][STEP]['R'] for PAIR in range(len(perms_jVSi))], axis=0)
                perms_mean = np.mean([perms_jVSi[PAIR][STEP]['R'] for PAIR in range(len(perms_jVSi))], axis=0)

            # run permutation tests
            groupData[COND][STEP]['R_median_pVals'], groupData[COND][STEP]['R_median_masks'], groupData[COND][STEP]['R_median_pSig'] = coupling_lm_permTest_v2(groupData[COND][STEP]['R_median'], perms_median, alpha)
            groupData[COND][STEP]['R_mean_pVals'], groupData[COND][STEP]['R_mean_masks'], groupData[COND][STEP]['R_mean_pSig'] = coupling_lm_permTest_v2(groupData[COND][STEP]['R_mean'], perms_mean, alpha)

            # feedback about group median R^2 values
            print('\n% of voxels where group MEDIAN (across pairs) mean (across speech turns within pairs) R^2 values is in top ' + str(round((alpha / 2) * 100,2)) + '% of null distribution')
            print('in step ' + str(STEP+1) + ' of the ' + storyConds[COND] + ' condition: ' + str(round(groupData[COND][STEP]['R_median_pSig']['two_above'] * 100,2)) + '%')
            print('\n% of voxels where group MEDIAN (across pairs) mean (across speech turns within pairs) R^2 values is in bottom ' + str(round((alpha / 2) * 100,2)) + '% of null distribution')
            print('in step ' + str(STEP+1) + ' of the ' + storyConds[COND] + ' condition: ' + str(round(groupData[COND][STEP]['R_median_pSig']['two_below'] * 100,2)) + '%')

            # feedback about group mean R^2 values
            print('\n% of voxels where group MEAN (across pairs) mean (across speech turns within pairs) R^2 values is in top ' + str(round((alpha / 2) * 100,2)) + '% of null distribution')
            print('in step ' + str(STEP+1) + ' of the ' + storyConds[COND] + ' condition: ' + str(round(groupData[COND][STEP]['R_mean_pSig']['two_above'] * 100,2)) + '%')
            print('\n% of voxels where group MEAN (across pairs) mean (across speech turns within pairs) R^2 values is in bottom ' + str(round((alpha / 2) * 100,2)) + '% of null distribution')
            print('in step ' + str(STEP+1) + ' of the ' + storyConds[COND] + ' condition: ' + str(round(groupData[COND][STEP]['R_mean_pSig']['two_below'] * 100,2)) + '%')

        # if using more than one modeling window...
        if numSteps > 1:

            # initialize dictionary
            groupData_paramAvg[COND] = {'R_median':np.empty(numVox),
                                        'R_median_pVals':dict(),
                                        'R_median_masks':dict(),
                                        'R_median_pSig':dict(),
                                        'R_mean':np.empty(numVox),
                                        'R_mean_pVals':dict(),
                                        'R_mean_masks':dict(),
                                        'R_mean_pSig':dict()}

            # if independent or joint condition...
            if COND < 2:

                # compute means and medians
                groupData_paramAvg[COND]['R_median'] = np.median([paramAvg[ROW]['R'] for ROW in condInds[COND]], axis=0)
                groupData_paramAvg[COND]['R_mean'] = np.mean([paramAvg[ROW]['R'] for ROW in condInds[COND]], axis=0)
                perms_median = np.median([perms_paramAvg[ROW]['R'] for ROW in condInds[COND]], axis=0)
                perms_mean = np.mean([perms_paramAvg[ROW]['R'] for ROW in condInds[COND]], axis=0)

            # if joint minus independent condition...
            else:

                # compute means and medians
                groupData_paramAvg[COND]['R_median'] = np.median([jVSi_paramAvg[PAIR]['R'] for PAIR in range(len(jVSi_paramAvg))], axis=0)
                groupData_paramAvg[COND]['R_mean'] = np.mean([jVSi_paramAvg[PAIR]['R'] for PAIR in range(len(jVSi_paramAvg))], axis=0)
                perms_median = np.median([perms_jVSi_paramAvg[PAIR]['R'] for ROW in range(len(perms_jVSi_paramAvg))], axis=0)
                perms_mean = np.mean([perms_jVSi_paramAvg[PAIR]['R'] for ROW in range(len(perms_jVSi_paramAvg))], axis=0)

            # run permutation tests
            groupData_paramAvg[COND]['R_median_pVals'], groupData_paramAvg[COND]['R_median_masks'], groupData_paramAvg[COND]['R_median_pSig'] = coupling_lm_permTest_v2(groupData_paramAvg[COND]['R_median'], perms_median, alpha)
            groupData_paramAvg[COND]['R_mean_pVals'], groupData_paramAvg[COND]['R_mean_masks'], groupData_paramAvg[COND]['R_mean_pSig'] = coupling_lm_permTest_v2(groupData_paramAvg[COND]['R_mean'], perms_mean, alpha)

            # feedback about group median R^2 values
            print('\n% of voxels where GROUP MEDIAN (across pairs) mean (across steps) mean (across speech turns within pairs) R^2 values is in top ' + str(round((alpha / 2) * 100,2)) + '% of null distribution')
            print('in step ' + str(STEP+1) + ' of the ' + storyConds[COND] + ' condition: ' + str(round(groupData_paramAvg[COND]['R_median_pSig']['two_above'] * 100,2)) + '%')
            print('\n% of voxels where GROUP MEDIAN (across pairs) mean (across steps) mean (across speech turns within pairs) R^2 values is in bottom ' + str(round((alpha / 2) * 100,2)) + '% of null distribution')
            print('in step ' + str(STEP+1) + ' of the ' + storyConds[COND] + ' condition: ' + str(round(groupData_paramAvg[COND]['R_median_pSig']['two_below'] * 100,2)) + '%')

            # feedback about group mean R^2 values
            print('\n% of voxels where GROUP MEAN (across pairs) mean (across steps) mean (across speech turns within pairs) R^2 values is in top ' + str(round((alpha / 2) * 100,2)) + '% of null distribution')
            print('in step ' + str(STEP+1) + ' of the ' + storyConds[COND] + ' condition: ' + str(round(groupData_paramAvg[COND]['R_mean_pSig']['two_above'] * 100,2)) + '%')
            print('\n% of voxels where GROUP MEAN (across pairs) mean (across steps) mean (across speech turns within pairs) R^2 values is in bottom ' + str(round((alpha / 2) * 100,2)) + '% of null distribution')
            print('in step ' + str(STEP+1) + ' of the ' + storyConds[COND] + ' condition: ' + str(round(groupData_paramAvg[COND]['R_mean_pSig']['two_below'] * 100,2)) + '%')

    ###########################################################
    ### store data from current voxel chunk in dictionaries ###
    ###########################################################

    # duration
    chunkDur[CHUNK] = {'lagComp_total':lagComp_total,
           'lagComp_mean_per_pairMap_row':lagComp_mean_per_pairMap_row}
    pairData_by_step[CHUNK] = iscData
    pairData_across_steps[CHUNK] = paramAvg
    pairData_jVSi_by_step[CHUNK] = jVSi
    pairData_jVSi_across_steps[CHUNK] = jVSi_paramAvg
    groupData_by_step[CHUNK] = groupData
    groupData_across_steps[CHUNK] = groupData_paramAvg

    # delete variables to save memory
    del speaker, listener, iscData, paramAvg, jVSi, jVSi_paramAvg, groupData, groupData_paramAvg, \
        perms, perms_paramAvg, perms_jVSi, perms_jVSi_paramAvg, perms_median, perms_mean


Running lag models with 2 pairs, excluding pseudo pairs
Number of voxel chunks: 2
Permutations: 10
Max TR lag: 2

*** CHUNK 1 OF 2: VOXELS 1 to 34940 ***

estimating lag model C1R1 of C2R8
DBIC participant: sid000007
CBS participant: hid000002
storytelling condition: independent
pair type: real
speaker: CBS

Step 1 of 2: TRs 0 to 19
% of voxels with signficant FDR-corrected R^2 values: 23.29%
Step 2 of 2: TRs 20 to 39
% of voxels with signficant FDR-corrected R^2 values: 22.01%

Across steps:
% of voxels with signficant FDR-corrected mean R^2 values: 22.66%
processing time for model 1: ~0.91 min

estimating lag model C1R2 of C2R8
DBIC participant: sid000007
CBS participant: hid000002
storytelling condition: independent
pair type: real
speaker: DBIC

Step 1 of 2: TRs 0 to 19
% of voxels with signficant FDR-corrected R^2 values: 18.34%
Step 2 of 2: TRs 20 to 39
% of voxels with signficant FDR-corrected R^2 values: 17.93%

Across steps:
% of voxels with signficant FDR-corrected mean R^2 

/dartfs-hpc/rc/home/z/f00589z/.conda/envs/hypeScanCentral/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3437: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  exec(code_obj, self.user_global_ns, self.user_ns)


In [11]:
# preallocate conditions arrays (1=independent, 2=joint, 3=joint-independent)
pairDataByStep = [[]] * len(storyConds)
groupDataByStep = [[]] * len(storyConds)
pairDataAcrossSteps = [[]] * len(storyConds)
groupDataAcrossSteps = [[]] * len(storyConds)

# for each condition (independent, joint)...
for COND in range(len(storyConds)):

    # preallocate pair arrays
    pairDataByStep[COND] = [[]] * len(condInds[0])
    if numSteps > 1:
        pairDataAcrossSteps[COND] = [[]] * len(condInds[0])

    # for each chunk of voxels...
    for CHUNK in range(numChunks):

        ########################################################
        ### pair-level mean parameter estimates across steps ###
        ########################################################

        if numSteps > 1:

            # for each set of pair data for a given condition...
            for PAIR in range(len(condInds[0])):

                # if on the first chunk...
                if CHUNK == 0:

                    # as long as we're not on the joint minus independent condition past the number of unique pairs...
                    if not (COND == 2 and PAIR >= uniquePairs.shape[0]):

                        # initialize across-steps dictionary
                        pairDataAcrossSteps[COND][PAIR] = {'betas':np.empty([totalVox,maxT*2+1]),
                                                           'resids':np.empty(totalVox),
                                                           'R':np.empty(totalVox),
                                                           'R_pVals':dict(),
                                                           'R_masks':dict(),
                                                           'R_pSig':dict()}
                        pairDataAcrossSteps[COND][PAIR]['R_pVals'] = {'right_tail':np.empty(totalVox),
                                                                      'left_tail':np.empty(totalVox),
                                                                      'two_tail':np.empty(totalVox),
                                                                      'map':np.zeros(totalVox, dtype=bool)}

                # if independent or joint condition...
                if COND < 2:

                    # get across-steps data
                    pairDataAcrossSteps[COND][PAIR]['betas'][chunkInds[CHUNK]] = pairData_across_steps[CHUNK][condInds[COND][PAIR]]['betas']
                    pairDataAcrossSteps[COND][PAIR]['resids'][chunkInds[CHUNK]] = np.squeeze(pairData_across_steps[CHUNK][condInds[COND][PAIR]]['resids'])
                    pairDataAcrossSteps[COND][PAIR]['R'][chunkInds[CHUNK]] = np.squeeze(pairData_across_steps[CHUNK][condInds[COND][PAIR]]['R'])
                    pairDataAcrossSteps[COND][PAIR]['R_pVals']['right_tail'][chunkInds[CHUNK]] = np.squeeze(pairData_across_steps[CHUNK][condInds[COND][PAIR]]['R_pVals']['right_tail'])
                    pairDataAcrossSteps[COND][PAIR]['R_pVals']['left_tail'][chunkInds[CHUNK]] = np.squeeze(pairData_across_steps[CHUNK][condInds[COND][PAIR]]['R_pVals']['left_tail'])
                    pairDataAcrossSteps[COND][PAIR]['R_pVals']['two_tail'][chunkInds[CHUNK]] = np.squeeze(pairData_across_steps[CHUNK][condInds[COND][PAIR]]['R_pVals']['two_tail'])
                    pairDataAcrossSteps[COND][PAIR]['R_pVals']['map'][chunkInds[CHUNK]] = np.squeeze(pairData_across_steps[CHUNK][condInds[COND][PAIR]]['R_pVals']['map'])

                # if joint minus independent condition...
                else:

                    # if we haven't already gone through the total number of unique pairs
                    if PAIR < uniquePairs.shape[0]:

                        # get across-steps data
                        pairDataAcrossSteps[COND][PAIR]['betas'][chunkInds[CHUNK]] = np.nan # presumably we're not interested in the difference between the betas
                        pairDataAcrossSteps[COND][PAIR]['resids'][chunkInds[CHUNK]] = np.nan # presumably we're not interested in the difference between the residuals
                        pairDataAcrossSteps[COND][PAIR]['R'][chunkInds[CHUNK]] = np.squeeze(pairData_jVSi_across_steps[CHUNK][PAIR]['R'])
                        pairDataAcrossSteps[COND][PAIR]['R_pVals']['right_tail'][chunkInds[CHUNK]] = np.squeeze(pairData_jVSi_across_steps[CHUNK][PAIR]['R_pVals']['right_tail'])
                        pairDataAcrossSteps[COND][PAIR]['R_pVals']['left_tail'][chunkInds[CHUNK]] = np.squeeze(pairData_jVSi_across_steps[CHUNK][PAIR]['R_pVals']['left_tail'])
                        pairDataAcrossSteps[COND][PAIR]['R_pVals']['two_tail'][chunkInds[CHUNK]] = np.squeeze(pairData_jVSi_across_steps[CHUNK][PAIR]['R_pVals']['two_tail'])
                        pairDataAcrossSteps[COND][PAIR]['R_pVals']['map'][chunkInds[CHUNK]] = np.squeeze(pairData_jVSi_across_steps[CHUNK][PAIR]['R_pVals']['map'])

                # if on the last chunk...
                if CHUNK == numChunks-1:

                    # as long as we're not on the joint minus independent condition past the number of unique pairs...
                    if not (COND == 2 and PAIR >= uniquePairs.shape[0]):

                        # reapply false discovery rate correction across all voxels
                        pairDataAcrossSteps[COND][PAIR]['R_pVals']['fdr_right'] = multi.fdrcorrection(pairDataAcrossSteps[COND][PAIR]['R_pVals']['right_tail'], alpha = alpha)
                        pairDataAcrossSteps[COND][PAIR]['R_pVals']['fdr_left'] = multi.fdrcorrection(pairDataAcrossSteps[COND][PAIR]['R_pVals']['left_tail'], alpha = alpha)
                        pairDataAcrossSteps[COND][PAIR]['R_pVals']['fdr_two'] = multi.fdrcorrection(pairDataAcrossSteps[COND][PAIR]['R_pVals']['two_tail'], alpha = alpha / 2)

                        # get R^2 masks where all voxels that did not survive FDR correction are set to zero
                        pairDataAcrossSteps[COND][PAIR]['R_masks']['right'] = np.copy(pairDataAcrossSteps[COND][PAIR]['R'])
                        pairDataAcrossSteps[COND][PAIR]['R_masks']['right'][np.invert(pairDataAcrossSteps[COND][PAIR]['R_pVals']['fdr_right'][0])] = 0
                        pairDataAcrossSteps[COND][PAIR]['R_masks']['left'] = np.copy(pairDataAcrossSteps[COND][PAIR]['R'])
                        pairDataAcrossSteps[COND][PAIR]['R_masks']['left'][np.invert(pairDataAcrossSteps[COND][PAIR]['R_pVals']['fdr_left'][0])] = 0
                        pairDataAcrossSteps[COND][PAIR]['R_masks']['two'] = np.copy(pairDataAcrossSteps[COND][PAIR]['R'])
                        pairDataAcrossSteps[COND][PAIR]['R_masks']['two'][np.invert(pairDataAcrossSteps[COND][PAIR]['R_pVals']['fdr_two'][0])] = 0
                        pairDataAcrossSteps[COND][PAIR]['R_masks']['two_above'] = np.copy(pairDataAcrossSteps[COND][PAIR]['R'])
                        pairDataAcrossSteps[COND][PAIR]['R_masks']['two_above'][np.minimum(pairDataAcrossSteps[COND][PAIR]['R_pVals']['fdr_two'][0], np.invert(pairDataAcrossSteps[COND][PAIR]['R_pVals']['map']))] = 0
                        pairDataAcrossSteps[COND][PAIR]['R_masks']['two_below'] = np.copy(pairDataAcrossSteps[COND][PAIR]['R'])
                        pairDataAcrossSteps[COND][PAIR]['R_masks']['two_below'][np.minimum(pairDataAcrossSteps[COND][PAIR]['R_pVals']['fdr_two'][0], pairDataAcrossSteps[COND][PAIR]['R_pVals']['map'])] = 0

                        # get some FDR corrected p-value summary info
                        pairDataAcrossSteps[COND][PAIR]['R_pSig']['right'] = len(np.where(pairDataAcrossSteps[COND][PAIR]['R_pVals']['fdr_right'][0])[0]) / totalVox
                        pairDataAcrossSteps[COND][PAIR]['R_pSig']['left'] = len(np.where(pairDataAcrossSteps[COND][PAIR]['R_pVals']['fdr_left'][0])[0]) / totalVox
                        pairDataAcrossSteps[COND][PAIR]['R_pSig']['two'] = len(np.where(pairDataAcrossSteps[COND][PAIR]['R_pVals']['fdr_two'][0])[0]) / totalVox
                        pairDataAcrossSteps[COND][PAIR]['R_pSig']['two_above'] = np.count_nonzero(np.minimum(pairDataAcrossSteps[COND][PAIR]['R_pVals']['fdr_two'][0], np.invert(pairDataAcrossSteps[COND][PAIR]['R_pVals']['map']))) / totalVox # proportion of voxels that show significant FDR CORRECTED R^2 values above the median
                        pairDataAcrossSteps[COND][PAIR]['R_pSig']['two_below'] = np.count_nonzero(np.minimum(pairDataAcrossSteps[COND][PAIR]['R_pVals']['fdr_two'][0], pairDataAcrossSteps[COND][PAIR]['R_pVals']['map'])) / totalVox

                        # feedback
                        print('\nDBIC sub ' + str(pairMap['dbicNum'][condInds[0][PAIR]]) + ', CBS sub ' + str(pairMap['cbsNum'][condInds[0][PAIR]]) + ', ' + storyConds[COND] + ' condition')
                        print('% of voxels wit`h signficant FDR-corrected mean (across steps) R^2 values: ' + str(round(pairDataAcrossSteps[COND][PAIR]['R_pSig']['two'] * 100,2)) + '%')

        else:

            pairDataAcrossSteps = np.nan

        ################################################
        ### pair-level step-wise parameter estimates ###
        ################################################

        # preallocate step array
        if CHUNK == 0:
            pairDataByStep[COND][PAIR] = [[]] * numSteps

        # for each rolling window step...
        for STEP in range(numSteps):

            # if it's the first chunk
            if CHUNK == 0:

                # initialize structure
                pairDataByStep[COND][PAIR][STEP] = {'betas':np.empty([totalVox,maxT*2+1]),
                                                    'resids':np.empty(totalVox),
                                                    'R':np.empty(totalVox),
                                                    'R_pVals':dict(),
                                                    'R_masks':dict(),
                                                    'R_pSig':dict()}
                pairDataByStep[COND][PAIR][STEP]['R_pVals'] = {'right_tail':np.empty(totalVox),
                                                              'left_tail':np.empty(totalVox),
                                                              'two_tail':np.empty(totalVox),
                                                              'map':np.zeros(totalVox, dtype=bool)}

            # if independent or joint condition...
            if COND < 2:

                pairDataByStep[COND][PAIR][STEP]['betas'][chunkInds[CHUNK],:] = pairData_by_step[CHUNK][condInds[COND][PAIR]][STEP]['betas']
                pairDataByStep[COND][PAIR][STEP]['resids'][chunkInds[CHUNK]] = np.squeeze(pairData_by_step[CHUNK][condInds[COND][PAIR]][STEP]['resids'])
                pairDataByStep[COND][PAIR][STEP]['R'][chunkInds[CHUNK]] = np.squeeze(pairData_by_step[CHUNK][condInds[COND][PAIR]][STEP]['R'])
                pairDataByStep[COND][PAIR][STEP]['R_pVals']['right_tail'][chunkInds[CHUNK]] = np.squeeze(pairData_by_step[CHUNK][condInds[COND][PAIR]][STEP]['R_pVals']['right_tail'])
                pairDataByStep[COND][PAIR][STEP]['R_pVals']['left_tail'][chunkInds[CHUNK]] = np.squeeze(pairData_by_step[CHUNK][condInds[COND][PAIR]][STEP]['R_pVals']['left_tail'])
                pairDataByStep[COND][PAIR][STEP]['R_pVals']['two_tail'][chunkInds[CHUNK]] = np.squeeze(pairData_by_step[CHUNK][condInds[COND][PAIR]][STEP]['R_pVals']['two_tail'])
                pairDataByStep[COND][PAIR][STEP]['R_pVals']['map'][chunkInds[CHUNK]] = np.squeeze(pairData_by_step[CHUNK][condInds[COND][PAIR]][STEP]['R_pVals']['map'])

            # if joint minus independent condition
            else:

                # if we haven't already gone through the total number of unique pairs
                if PAIR < uniquePairs.shape[0]:

                    pairDataByStep[COND][PAIR][STEP]['betas'][chunkInds[CHUNK]] = np.nan
                    pairDataByStep[COND][PAIR][STEP]['resids'][chunkInds[CHUNK]] = np.nan
                    pairDataByStep[COND][PAIR][STEP]['R'][chunkInds[CHUNK]] = np.squeeze(pairData_jVSi_by_step[CHUNK][PAIR][STEP]['R'])
                    pairDataByStep[COND][PAIR][STEP]['R_pVals']['right_tail'][chunkInds[CHUNK]] = np.squeeze(pairData_jVSi_by_step[CHUNK][PAIR][STEP]['R_pVals']['right_tail'])
                    pairDataByStep[COND][PAIR][STEP]['R_pVals']['left_tail'][chunkInds[CHUNK]] = np.squeeze(pairData_jVSi_by_step[CHUNK][PAIR][STEP]['R_pVals']['left_tail'])
                    pairDataByStep[COND][PAIR][STEP]['R_pVals']['two_tail'][chunkInds[CHUNK]] = np.squeeze(pairData_jVSi_by_step[CHUNK][PAIR][STEP]['R_pVals']['two_tail'])
                    pairDataByStep[COND][PAIR][STEP]['R_pVals']['map'][chunkInds[CHUNK]] = np.squeeze(pairData_jVSi_by_step[CHUNK][PAIR][STEP]['R_pVals']['map'])

            # if on the last chunk...
            if CHUNK == numChunks-1:

                # as long as we're not on the joint minus independent condition past the number of unique pairs...
                if not (COND == 2 and PAIR >= uniquePairs.shape[0]):

                    # reapply false discovery rate correction across all voxels
                    pairDataByStep[COND][PAIR][STEP]['R_pVals']['fdr_right'] = multi.fdrcorrection(pairDataByStep[COND][PAIR][STEP]['R_pVals']['right_tail'], alpha = alpha)
                    pairDataByStep[COND][PAIR][STEP]['R_pVals']['fdr_left'] = multi.fdrcorrection(pairDataByStep[COND][PAIR][STEP]['R_pVals']['left_tail'], alpha = alpha)
                    pairDataByStep[COND][PAIR][STEP]['R_pVals']['fdr_two'] = multi.fdrcorrection(pairDataByStep[COND][PAIR][STEP]['R_pVals']['two_tail'], alpha = alpha / 2)

                    # get R^2 masks where all voxels that did not survive FDR correction are set to zero
                    pairDataByStep[COND][PAIR][STEP]['R_masks']['right'] = np.copy(pairDataByStep[COND][PAIR][STEP]['R'])
                    pairDataByStep[COND][PAIR][STEP]['R_masks']['right'][np.invert(pairDataByStep[COND][PAIR][STEP]['R_pVals']['fdr_right'][0])] = 0
                    pairDataByStep[COND][PAIR][STEP]['R_masks']['left'] = np.copy(pairDataByStep[COND][PAIR][STEP]['R'])
                    pairDataByStep[COND][PAIR][STEP]['R_masks']['left'][np.invert(pairDataByStep[COND][PAIR][STEP]['R_pVals']['fdr_left'][0])] = 0
                    pairDataByStep[COND][PAIR][STEP]['R_masks']['two'] = np.copy(pairDataByStep[COND][PAIR][STEP]['R'])
                    pairDataByStep[COND][PAIR][STEP]['R_masks']['two'][np.invert(pairDataByStep[COND][PAIR][STEP]['R_pVals']['fdr_two'][0])] = 0
                    pairDataByStep[COND][PAIR][STEP]['R_masks']['two_above'] = np.copy(pairDataByStep[COND][PAIR][STEP]['R'])
                    pairDataByStep[COND][PAIR][STEP]['R_masks']['two_above'][np.minimum(pairDataByStep[COND][PAIR][STEP]['R_pVals']['fdr_two'][0], np.invert(pairDataByStep[COND][PAIR][STEP]['R_pVals']['map']))] = 0
                    pairDataByStep[COND][PAIR][STEP]['R_masks']['two_below'] = np.copy(pairDataByStep[COND][PAIR][STEP]['R'])
                    pairDataByStep[COND][PAIR][STEP]['R_masks']['two_below'][np.minimum(pairDataByStep[COND][PAIR][STEP]['R_pVals']['fdr_two'][0], pairDataByStep[COND][PAIR][STEP]['R_pVals']['map'])] = 0

                    # get some FDR corrected p-value summary info
                    pairDataByStep[COND][PAIR][STEP]['R_pSig']['right'] = len(np.where(pairDataByStep[COND][PAIR][STEP]['R_pVals']['fdr_right'][0])[0]) / totalVox
                    pairDataByStep[COND][PAIR][STEP]['R_pSig']['left'] = len(np.where(pairDataByStep[COND][PAIR][STEP]['R_pVals']['fdr_left'][0])[0]) / totalVox
                    pairDataByStep[COND][PAIR][STEP]['R_pSig']['two'] = len(np.where(pairDataByStep[COND][PAIR][STEP]['R_pVals']['fdr_two'][0])[0]) / totalVox
                    pairDataByStep[COND][PAIR][STEP]['R_pSig']['two_above'] = np.count_nonzero(np.minimum(pairDataByStep[COND][PAIR][STEP]['R_pVals']['fdr_two'][0], np.invert(pairDataByStep[COND][PAIR][STEP]['R_pVals']['map']))) / totalVox # proportion of voxels that show significant FDR CORRECTED R^2 values above the median
                    pairDataByStep[COND][PAIR][STEP]['R_pSig']['two_below'] = np.count_nonzero(np.minimum(pairDataByStep[COND][PAIR][STEP]['R_pVals']['fdr_two'][0], pairDataByStep[COND][PAIR][STEP]['R_pVals']['map'])) / totalVox

                    # feedback
                    print('\nDBIC sub ' + str(pairMap['dbicNum'][condInds[0][PAIR]]) + ', CBS sub ' + str(pairMap['cbsNum'][condInds[0][PAIR]]) + ', ' + storyConds[COND] + ' condition, step ' + str(STEP+1))
                    print('% of voxels wit`h signficant FDR-corrected mean (across steps) R^2 values: ' + str(round(pairDataByStep[COND][PAIR][STEP]['R_pSig']['two'] * 100,2)) + '%')

        ###############################
        ### group data across steps ###
        ###############################

        if numSteps > 1:

            # if it's the first chunk
            if CHUNK == 0:

                # preallocate across-steps data dictionaries
                groupDataAcrossSteps[COND] = {'R_median':np.empty(totalVox),
                                              'R_median_pVals':dict(),
                                              'R_median_masks':dict(),
                                              'R_median_pSig':dict(),
                                              'R_mean':np.empty(totalVox),
                                              'R_mean_pVals':dict(),
                                              'R_mean_masks':dict(),
                                              'R_mean_pSig':dict()}
                groupDataAcrossSteps[COND]['R_median_pVals'] = {'right_tail':np.empty(totalVox),
                                                                'left_tail':np.empty(totalVox),
                                                                'two_tail':np.empty(totalVox),
                                                                'map':np.zeros(totalVox, dtype=bool)}
                groupDataAcrossSteps[COND]['R_mean_pVals'] = {'right_tail':np.empty(totalVox),
                                                              'left_tail':np.empty(totalVox),
                                                              'two_tail':np.empty(totalVox),
                                                              'map':np.zeros(totalVox, dtype=bool)}

            # get group data by chunk
            groupDataAcrossSteps[COND]['R_median'][chunkInds[CHUNK]] = np.squeeze(groupData_across_steps[CHUNK][COND]['R_median'])
            groupDataAcrossSteps[COND]['R_median_pVals']['right_tail'][chunkInds[CHUNK]] = np.squeeze(groupData_across_steps[CHUNK][COND]['R_median_pVals']['right_tail'])
            groupDataAcrossSteps[COND]['R_median_pVals']['left_tail'][chunkInds[CHUNK]] = np.squeeze(groupData_across_steps[CHUNK][COND]['R_median_pVals']['left_tail'])
            groupDataAcrossSteps[COND]['R_median_pVals']['two_tail'][chunkInds[CHUNK]] = np.squeeze(groupData_across_steps[CHUNK][COND]['R_median_pVals']['two_tail'])
            groupDataAcrossSteps[COND]['R_median_pVals']['map'][chunkInds[CHUNK]] = np.squeeze(groupData_across_steps[CHUNK][COND]['R_median_pVals']['map'])
            groupDataAcrossSteps[COND]['R_mean'][chunkInds[CHUNK]] = np.squeeze(groupData_across_steps[CHUNK][COND]['R_mean'])
            groupDataAcrossSteps[COND]['R_mean_pVals']['right_tail'][chunkInds[CHUNK]] = np.squeeze(groupData_across_steps[CHUNK][COND]['R_mean_pVals']['right_tail'])
            groupDataAcrossSteps[COND]['R_mean_pVals']['left_tail'][chunkInds[CHUNK]] = np.squeeze(groupData_across_steps[CHUNK][COND]['R_mean_pVals']['left_tail'])
            groupDataAcrossSteps[COND]['R_mean_pVals']['two_tail'][chunkInds[CHUNK]] = np.squeeze(groupData_across_steps[CHUNK][COND]['R_mean_pVals']['two_tail'])
            groupDataAcrossSteps[COND]['R_mean_pVals']['map'][chunkInds[CHUNK]] = np.squeeze(groupData_across_steps[CHUNK][COND]['R_mean_pVals']['map'])

            # if it's the last chunk
            if CHUNK == numChunks-1:

                # reapply FDR correction to p-values
                groupDataAcrossSteps[COND]['R_median_pVals']['fdr_right'] = multi.fdrcorrection(groupDataAcrossSteps[COND]['R_median_pVals']['right_tail'], alpha = alpha)
                groupDataAcrossSteps[COND]['R_median_pVals']['fdr_left'] = multi.fdrcorrection(groupDataAcrossSteps[COND]['R_median_pVals']['left_tail'], alpha = alpha)
                groupDataAcrossSteps[COND]['R_median_pVals']['fdr_two'] = multi.fdrcorrection(groupDataAcrossSteps[COND]['R_median_pVals']['two_tail'], alpha = alpha / 2)
                groupDataAcrossSteps[COND]['R_mean_pVals']['fdr_right'] = multi.fdrcorrection(groupDataAcrossSteps[COND]['R_mean_pVals']['right_tail'], alpha = alpha)
                groupDataAcrossSteps[COND]['R_mean_pVals']['fdr_left'] = multi.fdrcorrection(groupDataAcrossSteps[COND]['R_mean_pVals']['left_tail'], alpha = alpha)
                groupDataAcrossSteps[COND]['R_mean_pVals']['fdr_two'] = multi.fdrcorrection(groupDataAcrossSteps[COND]['R_mean_pVals']['two_tail'], alpha = alpha / 2)

                # get R^2 masks where all voxels that did not survive FDR correction are set to zero
                groupDataAcrossSteps[COND]['R_median_masks']['right'] = np.copy(groupDataAcrossSteps[COND]['R_median'])
                groupDataAcrossSteps[COND]['R_median_masks']['right'][np.invert(groupDataAcrossSteps[COND]['R_median_pVals']['fdr_right'][0])] = 0
                groupDataAcrossSteps[COND]['R_median_masks']['left'] = np.copy(groupDataAcrossSteps[COND]['R_median'])
                groupDataAcrossSteps[COND]['R_median_masks']['left'][np.invert(groupDataAcrossSteps[COND]['R_median_pVals']['fdr_left'][0])] = 0
                groupDataAcrossSteps[COND]['R_median_masks']['two'] = np.copy(groupDataAcrossSteps[COND]['R_median'])
                groupDataAcrossSteps[COND]['R_median_masks']['two'][np.invert(groupDataAcrossSteps[COND]['R_median_pVals']['fdr_two'][0])] = 0
                groupDataAcrossSteps[COND]['R_median_masks']['two_above'] = np.copy(groupDataAcrossSteps[COND]['R_median'])
                groupDataAcrossSteps[COND]['R_median_masks']['two_above'][np.minimum(groupDataAcrossSteps[COND]['R_median_pVals']['fdr_two'][0], np.invert(groupDataAcrossSteps[COND]['R_median_pVals']['map']))] = 0
                groupDataAcrossSteps[COND]['R_median_masks']['two_below'] = np.copy(groupDataAcrossSteps[COND]['R_median'])
                groupDataAcrossSteps[COND]['R_median_masks']['two_below'][np.minimum(groupDataAcrossSteps[COND]['R_median_pVals']['fdr_two'][0], groupDataAcrossSteps[COND]['R_median_pVals']['map'])] = 0
                groupDataAcrossSteps[COND]['R_mean_masks']['right'] = np.copy(groupDataAcrossSteps[COND]['R_mean'])
                groupDataAcrossSteps[COND]['R_mean_masks']['right'][np.invert(groupDataAcrossSteps[COND]['R_mean_pVals']['fdr_right'][0])] = 0
                groupDataAcrossSteps[COND]['R_mean_masks']['left'] = np.copy(groupDataAcrossSteps[COND]['R_mean'])
                groupDataAcrossSteps[COND]['R_mean_masks']['left'][np.invert(groupDataAcrossSteps[COND]['R_mean_pVals']['fdr_left'][0])] = 0
                groupDataAcrossSteps[COND]['R_mean_masks']['two'] = np.copy(groupDataAcrossSteps[COND]['R_mean'])
                groupDataAcrossSteps[COND]['R_mean_masks']['two'][np.invert(groupDataAcrossSteps[COND]['R_mean_pVals']['fdr_two'][0])] = 0
                groupDataAcrossSteps[COND]['R_mean_masks']['two_above'] = np.copy(groupDataAcrossSteps[COND]['R_mean'])
                groupDataAcrossSteps[COND]['R_mean_masks']['two_above'][np.minimum(groupDataAcrossSteps[COND]['R_mean_pVals']['fdr_two'][0], np.invert(groupDataAcrossSteps[COND]['R_mean_pVals']['map']))] = 0
                groupDataAcrossSteps[COND]['R_mean_masks']['two_below'] = np.copy(groupDataAcrossSteps[COND]['R_mean'])
                groupDataAcrossSteps[COND]['R_mean_masks']['two_below'][np.minimum(groupDataAcrossSteps[COND]['R_mean_pVals']['fdr_two'][0], groupDataAcrossSteps[COND]['R_mean_pVals']['map'])] = 0

                # get some FDR corrected p-value summary info
                groupDataAcrossSteps[COND]['R_median_pSig']['right'] = len(np.where(groupDataAcrossSteps[COND]['R_median_pVals']['fdr_right'][0])[0]) / totalVox
                groupDataAcrossSteps[COND]['R_median_pSig']['left'] = len(np.where(groupDataAcrossSteps[COND]['R_median_pVals']['fdr_left'][0])[0]) / totalVox
                groupDataAcrossSteps[COND]['R_median_pSig']['two'] = len(np.where(groupDataAcrossSteps[COND]['R_median_pVals']['fdr_two'][0])[0]) / totalVox
                groupDataAcrossSteps[COND]['R_median_pSig']['two_above'] = np.count_nonzero(np.minimum(groupDataAcrossSteps[COND]['R_median_pVals']['fdr_two'][0], np.invert(groupDataAcrossSteps[COND]['R_median_pVals']['map']))) / totalVox # proportion of voxels that show significant FDR CORRECTED R^2 values above the median
                groupDataAcrossSteps[COND]['R_median_pSig']['two_below'] = np.count_nonzero(np.minimum(groupDataAcrossSteps[COND]['R_median_pVals']['fdr_two'][0], groupDataAcrossSteps[COND]['R_median_pVals']['map'])) / totalVox
                groupDataAcrossSteps[COND]['R_mean_pSig']['right'] = len(np.where(groupDataAcrossSteps[COND]['R_mean_pVals']['fdr_right'][0])[0]) / totalVox
                groupDataAcrossSteps[COND]['R_mean_pSig']['left'] = len(np.where(groupDataAcrossSteps[COND]['R_mean_pVals']['fdr_left'][0])[0]) / totalVox
                groupDataAcrossSteps[COND]['R_mean_pSig']['two'] = len(np.where(groupDataAcrossSteps[COND]['R_mean_pVals']['fdr_two'][0])[0]) / totalVox
                groupDataAcrossSteps[COND]['R_mean_pSig']['two_above'] = np.count_nonzero(np.minimum(groupDataAcrossSteps[COND]['R_mean_pVals']['fdr_two'][0], np.invert(groupDataAcrossSteps[COND]['R_mean_pVals']['map']))) / totalVox # proportion of voxels that show significant FDR CORRECTED R^2 values above the median
                groupDataAcrossSteps[COND]['R_mean_pSig']['two_below'] = np.count_nonzero(np.minimum(groupDataAcrossSteps[COND]['R_mean_pVals']['fdr_two'][0], groupDataAcrossSteps[COND]['R_mean_pVals']['map'])) / totalVox

                # feedback
                print('\nGroup data, ' + storyConds[COND] + ' condition')
                print('% of voxels with signficant FDR-corrected MEDIAN (across pairs) mean (across steps) R^2 values: ' + str(round(groupDataAcrossSteps[COND]['R_median_pSig']['two'] * 100,2)) + '%')
                print('% of voxels with signficant FDR-corrected MEAN (across pairs) mean (across steps) R^2 values: ' + str(round(groupDataAcrossSteps[COND]['R_mean_pSig']['two'] * 100,2)) + '%')

        else:

            groupDataAcrossSteps = np.nan

        #################################################
        ### group-level step-wise parameter estimates ###
        #################################################

        # preallocate step array
        if CHUNK == 0:
            groupDataByStep[COND] = [[]] * numSteps

        # for each rolling window step...
        for STEP in range(numSteps):

            # if it's the first chunk
            if CHUNK == 0:

                # initialize structure
                groupDataByStep[COND][STEP] = {'R_median':np.empty(totalVox),
                                               'R_median_pVals':dict(),
                                               'R_median_masks':dict(),
                                               'R_median_pSig':dict(),
                                               'R_mean':np.empty(totalVox),
                                               'R_mean_pVals':dict(),
                                               'R_mean_masks':dict(),
                                               'R_mean_pSig':dict()}
                groupDataByStep[COND][STEP]['R_median_pVals'] = {'right_tail':np.empty(totalVox),
                                                                 'left_tail':np.empty(totalVox),
                                                                 'two_tail':np.empty(totalVox),
                                                                 'map':np.zeros(totalVox, dtype=bool)}
                groupDataByStep[COND][STEP]['R_mean_pVals'] = {'right_tail':np.empty(totalVox),
                                                               'left_tail':np.empty(totalVox),
                                                               'two_tail':np.empty(totalVox),
                                                               'map':np.zeros(totalVox, dtype=bool)}
                # groupDataByStep[COND][STEP]['R_median_masks'] = {}
                # groupDataByStep[COND][STEP]['R_mean_masks'] = {}

            # get group data by step
            groupDataByStep[COND][STEP]['R_median'][chunkInds[CHUNK]] = np.squeeze(groupData_by_step[CHUNK][COND][STEP]['R_median'])
            groupDataByStep[COND][STEP]['R_median_pVals']['right_tail'][chunkInds[CHUNK]] = np.squeeze(groupData_by_step[CHUNK][COND][STEP]['R_median_pVals']['right_tail'])
            groupDataByStep[COND][STEP]['R_median_pVals']['left_tail'][chunkInds[CHUNK]] = np.squeeze(groupData_by_step[CHUNK][COND][STEP]['R_median_pVals']['left_tail'])
            groupDataByStep[COND][STEP]['R_median_pVals']['two_tail'][chunkInds[CHUNK]] = np.squeeze(groupData_by_step[CHUNK][COND][STEP]['R_median_pVals']['two_tail'])
            groupDataByStep[COND][STEP]['R_median_pVals']['map'][chunkInds[CHUNK]] = np.squeeze(groupData_by_step[CHUNK][COND][STEP]['R_median_pVals']['map'])
            groupDataByStep[COND][STEP]['R_mean'][chunkInds[CHUNK]] = np.squeeze(groupData_by_step[CHUNK][COND][STEP]['R_mean'])
            groupDataByStep[COND][STEP]['R_mean_pVals']['right_tail'][chunkInds[CHUNK]] = np.squeeze(groupData_by_step[CHUNK][COND][STEP]['R_mean_pVals']['right_tail'])
            groupDataByStep[COND][STEP]['R_mean_pVals']['left_tail'][chunkInds[CHUNK]] = np.squeeze(groupData_by_step[CHUNK][COND][STEP]['R_mean_pVals']['left_tail'])
            groupDataByStep[COND][STEP]['R_mean_pVals']['two_tail'][chunkInds[CHUNK]] = np.squeeze(groupData_by_step[CHUNK][COND][STEP]['R_mean_pVals']['two_tail'])
            groupDataByStep[COND][STEP]['R_mean_pVals']['map'][chunkInds[CHUNK]] = np.squeeze(groupData_by_step[CHUNK][COND][STEP]['R_mean_pVals']['map'])

            # if on the last chunk...
            if CHUNK == numChunks-1:

                # reapply false discovery rate correction across all voxels
                groupDataByStep[COND][STEP]['R_median_pVals']['fdr_right'] = multi.fdrcorrection(groupDataByStep[COND][STEP]['R_median_pVals']['right_tail'], alpha = alpha)
                groupDataByStep[COND][STEP]['R_median_pVals']['fdr_left'] = multi.fdrcorrection(groupDataByStep[COND][STEP]['R_median_pVals']['left_tail'], alpha = alpha)
                groupDataByStep[COND][STEP]['R_median_pVals']['fdr_two'] = multi.fdrcorrection(groupDataByStep[COND][STEP]['R_median_pVals']['two_tail'], alpha = alpha / 2)
                groupDataByStep[COND][STEP]['R_mean_pVals']['fdr_right'] = multi.fdrcorrection(groupDataByStep[COND][STEP]['R_mean_pVals']['right_tail'], alpha = alpha)
                groupDataByStep[COND][STEP]['R_mean_pVals']['fdr_left'] = multi.fdrcorrection(groupDataByStep[COND][STEP]['R_mean_pVals']['left_tail'], alpha = alpha)
                groupDataByStep[COND][STEP]['R_mean_pVals']['fdr_two'] = multi.fdrcorrection(groupDataByStep[COND][STEP]['R_mean_pVals']['two_tail'], alpha = alpha / 2)

                # get R^2 masks where all voxels that did not survive FDR correction are set to zero
                groupDataByStep[COND][STEP]['R_median_masks']['right'] = np.copy(groupDataByStep[COND][STEP]['R_median'])
                groupDataByStep[COND][STEP]['R_median_masks']['right'][np.invert(groupDataByStep[COND][STEP]['R_median_pVals']['fdr_right'][0])] = 0
                groupDataByStep[COND][STEP]['R_median_masks']['left'] = np.copy(groupDataByStep[COND][STEP]['R_median'])
                groupDataByStep[COND][STEP]['R_median_masks']['left'][np.invert(groupDataByStep[COND][STEP]['R_median_pVals']['fdr_left'][0])] = 0
                groupDataByStep[COND][STEP]['R_median_masks']['two'] = np.copy(groupDataByStep[COND][STEP]['R_median'])
                groupDataByStep[COND][STEP]['R_median_masks']['two'][np.invert(groupDataByStep[COND][STEP]['R_median_pVals']['fdr_two'][0])] = 0
                groupDataByStep[COND][STEP]['R_median_masks']['two_above'] = np.copy(groupDataByStep[COND][STEP]['R_median'])
                groupDataByStep[COND][STEP]['R_median_masks']['two_above'][np.minimum(groupDataByStep[COND][STEP]['R_median_pVals']['fdr_two'][0], np.invert(groupDataByStep[COND][STEP]['R_median_pVals']['map']))] = 0
                groupDataByStep[COND][STEP]['R_median_masks']['two_below'] = np.copy(groupDataByStep[COND][STEP]['R_median'])
                groupDataByStep[COND][STEP]['R_median_masks']['two_below'][np.minimum(groupDataByStep[COND][STEP]['R_median_pVals']['fdr_two'][0], groupDataByStep[COND][STEP]['R_median_pVals']['map'])] = 0
                groupDataByStep[COND][STEP]['R_mean_masks']['right'] = np.copy(groupDataByStep[COND][STEP]['R_mean'])
                groupDataByStep[COND][STEP]['R_mean_masks']['right'][np.invert(groupDataByStep[COND][STEP]['R_mean_pVals']['fdr_right'][0])] = 0
                groupDataByStep[COND][STEP]['R_mean_masks']['left'] = np.copy(groupDataByStep[COND][STEP]['R_mean'])
                groupDataByStep[COND][STEP]['R_mean_masks']['left'][np.invert(groupDataByStep[COND][STEP]['R_mean_pVals']['fdr_left'][0])] = 0
                groupDataByStep[COND][STEP]['R_mean_masks']['two'] = np.copy(groupDataByStep[COND][STEP]['R_mean'])
                groupDataByStep[COND][STEP]['R_mean_masks']['two'][np.invert(groupDataByStep[COND][STEP]['R_mean_pVals']['fdr_two'][0])] = 0
                groupDataByStep[COND][STEP]['R_mean_masks']['two_above'] = np.copy(groupDataByStep[COND][STEP]['R_mean'])
                groupDataByStep[COND][STEP]['R_mean_masks']['two_above'][np.minimum(groupDataByStep[COND][STEP]['R_mean_pVals']['fdr_two'][0], np.invert(groupDataByStep[COND][STEP]['R_mean_pVals']['map']))] = 0
                groupDataByStep[COND][STEP]['R_mean_masks']['two_below'] = np.copy(groupDataByStep[COND][STEP]['R_mean'])
                groupDataByStep[COND][STEP]['R_mean_masks']['two_below'][np.minimum(groupDataByStep[COND][STEP]['R_mean_pVals']['fdr_two'][0], groupDataByStep[COND][STEP]['R_mean_pVals']['map'])] = 0

                # get some FDR corrected p-value summary info
                groupDataByStep[COND][STEP]['R_median_pSig']['right'] = len(np.where(groupDataByStep[COND][STEP]['R_median_pVals']['fdr_right'][0])[0]) / totalVox
                groupDataByStep[COND][STEP]['R_median_pSig']['left'] = len(np.where(groupDataByStep[COND][STEP]['R_median_pVals']['fdr_left'][0])[0]) / totalVox
                groupDataByStep[COND][STEP]['R_median_pSig']['two'] = len(np.where(groupDataByStep[COND][STEP]['R_median_pVals']['fdr_two'][0])[0]) / totalVox
                groupDataByStep[COND][STEP]['R_median_pSig']['two_above'] = np.count_nonzero(np.minimum(groupDataByStep[COND][STEP]['R_median_pVals']['fdr_two'][0], np.invert(groupDataByStep[COND][STEP]['R_median_pVals']['map']))) / totalVox # proportion of voxels that show significant FDR CORRECTED R^2 values above the median
                groupDataByStep[COND][STEP]['R_median_pSig']['two_below'] = np.count_nonzero(np.minimum(groupDataByStep[COND][STEP]['R_median_pVals']['fdr_two'][0], groupDataByStep[COND][STEP]['R_median_pVals']['map'])) / totalVox
                groupDataByStep[COND][STEP]['R_mean_pSig']['right'] = len(np.where(groupDataByStep[COND][STEP]['R_mean_pVals']['fdr_right'][0])[0]) / totalVox
                groupDataByStep[COND][STEP]['R_mean_pSig']['left'] = len(np.where(groupDataByStep[COND][STEP]['R_mean_pVals']['fdr_left'][0])[0]) / totalVox
                groupDataByStep[COND][STEP]['R_mean_pSig']['two'] = len(np.where(groupDataByStep[COND][STEP]['R_mean_pVals']['fdr_two'][0])[0]) / totalVox
                groupDataByStep[COND][STEP]['R_mean_pSig']['two_above'] = np.count_nonzero(np.minimum(groupDataByStep[COND][STEP]['R_mean_pVals']['fdr_two'][0], np.invert(groupDataByStep[COND][STEP]['R_mean_pVals']['map']))) / totalVox # proportion of voxels that show significant FDR CORRECTED R^2 values above the median
                groupDataByStep[COND][STEP]['R_mean_pSig']['two_below'] = np.count_nonzero(np.minimum(groupDataByStep[COND][STEP]['R_mean_pVals']['fdr_two'][0], groupDataByStep[COND][STEP]['R_mean_pVals']['map'])) / totalVox

                # feedback
                print('\nGroup data, ' + storyConds[COND] + ' condition, step ' + str(STEP+1))
                print('% of voxels with signficant FDR-corrected MEDIAN (across pairs) mean (across steps) R^2 values: ' + str(round(groupDataByStep[COND][STEP]['R_median_pSig']['two'] * 100,2)) + '%')
                print('% of voxels with signficant FDR-corrected MEAN (across pairs) mean (across steps) R^2 values: ' + str(round(groupDataByStep[COND][STEP]['R_mean_pSig']['two'] * 100,2)) + '%')


DBIC sub 2, CBS sub 2, independent condition
% of voxels wit`h signficant FDR-corrected mean (across steps) R^2 values: 21.66%

DBIC sub 2, CBS sub 2, independent condition
% of voxels wit`h signficant FDR-corrected mean (across steps) R^2 values: 16.23%

DBIC sub 3, CBS sub 3, independent condition
% of voxels wit`h signficant FDR-corrected mean (across steps) R^2 values: 15.71%

DBIC sub 3, CBS sub 3, independent condition
% of voxels wit`h signficant FDR-corrected mean (across steps) R^2 values: 18.42%

DBIC sub 3, CBS sub 3, independent condition, step 1
% of voxels wit`h signficant FDR-corrected mean (across steps) R^2 values: 17.93%

DBIC sub 3, CBS sub 3, independent condition, step 2
% of voxels wit`h signficant FDR-corrected mean (across steps) R^2 values: 19.47%

Group data, independent condition
% of voxels with signficant FDR-corrected MEDIAN (across pairs) mean (across steps) R^2 values: 17.88%
% of voxels with signficant FDR-corrected MEAN (across pairs) mean (across ste

In [12]:
totalDur = round((time.time() - startTime) / 60,2) # [min]
print('\ntotal duration: ~' + str(totalDur) + ' min')


total duration: ~3.05 min


In [13]:
duration = {'chunkDur':chunkDur,
            'totalDur':totalDur}

In [14]:
if saveOutput:

    # set save file name -- in .py file just save a parameters variable so you dont have to make the
    # save name wildly long...
    saveName = 'lagModel_mode' + str(rollMode)
    if rollMode == 3:
        saveName = saveName + '_winTRs' + str(winTRs) + '_stepSize' + str(stepSize) + '_numSteps' + str(numSteps)
    saveName = saveName + '_shuffRange' + str(shuffleRange) + '_perms' + str(permuts) + '_pad' + padding
    if circShift:
        saveName = saveName + '_cShift'
    else:
        saveName = saveName + '_pScram'
    if pseudo:
        saveName = saveName + '_pseudo'
    print('saving output...')
    saveFile = outputFolder + saveName + '.pkl'
    with open(saveFile,'wb') as f:
            pickle.dump([rngSeed, pairMap, chunkInds, pairDataByStep, groupDataByStep, pairDataAcrossSteps, groupDataAcrossSteps, duration], f, protocol=4)
    print('output saved here: ' + saveFile)

saving output...
output saved here: /dartfs-hpc/rc/lab/W/WheatleyT/f00589z/hyperscanning/storytelling/lag_model_output/lagModel_mode0_shuffRange0_perms10_padmean_pScram.pkl


In [ ]:
print('\n\n\nThe loony bin is closing shop, greetings',
          'from The Combine!\n\n')

In [ ]:
def surfaceStatMap(masker,statMapVec,avgSurface,thresh=0, conditions=['']):

    """
    :param masker:
    :param statMapVec:
    :param avgSurface:
    :param thresh:
    :param conditions: array of strings corresponding to different conditions
    :return:
    """

    # preallocate task arrays
    statMap = [[]] * len(conditions)
    texture = [[]] * len(conditions)
    view = [[]] * len(conditions)

    # for each task...
    for COND in range(len(conditions)):

        # get stat map
        statMap[COND] = masker.inverse_transform(statMapVec[COND])

        # surface plot
        texture[COND] = [[]] * 2
        view[COND] = [[]] * 2

        for HEMI in [0,1]:
            if HEMI == 0:
                texture[COND][HEMI] = surface.vol_to_surf(statMap[COND], avgSurface.pial_left)
                view[COND][HEMI] = plotting.view_surf(avgSurface.infl_left,
                                                                       texture[COND][HEMI],
                                                                       threshold=thresh,
                                                                       colorbar=True,
                                                                       title= conditions[COND] + ' left',
                                                                       bg_map=avgSurface.sulc_left)
            else:
                texture[COND][HEMI] = surface.vol_to_surf(statMap[COND], avgSurface.pial_right)
                view[COND][HEMI] = plotting.view_surf(avgSurface.infl_right,
                                                                       texture[COND][HEMI],
                                                                       threshold=thresh,
                                                                       colorbar=True,
                                                                       title=conditions[COND] + ', right',
                                                                       bg_map=avgSurface.sulc_right)

    return view

In [ ]:
# get masker object and average surface mesh
maskFile = '/dartfs-hpc/rc/home/z/f00589z/hyperscanning/control_tasks/nuisRegr_input_files/mni_asym09c_mask_resamp3x3.nii.gz'
maskImg = nImage.load_img(maskFile)
masker = input_data.NiftiMasker(maskImg)
masker.fit_transform(maskImg)
fsaverage = datasets.fetch_surf_fsaverage()

# reformat data for stat map function and get joint - ind stat map vector
statVecs = [[]] * 3
STEP = 0
for COND in range(len(statVecs)):
    statVecs[COND] = groupDataAcrossSteps[COND]['R_median_masks']['two']

# make stat maps
view = surfaceStatMap(masker,statVecs,fsaverage,0.001,['independent','joint','joint - independent'])

In [ ]:
view[0][0]

In [ ]:
view[0][1]

In [ ]:
view[1][0]

In [ ]:
view[1][1]

In [ ]:
view[2][0]

In [ ]:
view[2][1]
